In [1]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
from linearmodels import IV2SLS, IVGMM
import numpy as np
import scipy.stats as st

In [2]:
treated_df = pd.read_csv("ps7data_treated.csv", sep="  ", names=["treated", "age", "education", "black", "hispanic","married","nodegree","re75","re78"], engine="python")
control_df = pd.read_csv("ps7data_control.csv", sep="  ", names=["treated", "age", "education", "black", "hispanic","married","nodegree","re75","re78"], engine="python")

In [3]:
df = control_df.append(treated_df)
df["constant"] = 1
df

,treated,age,education,black,hispanic,married,nodegree,re75,re78,constant
0,0.0,23.0,10.0,1.0,0.0,0.0,1.0,0.000,0.000,1
1,0.0,26.0,12.0,0.0,0.0,0.0,0.0,0.000,12383.680,1
2,0.0,22.0,9.0,1.0,0.0,0.0,1.0,0.000,0.000,1
3,0.0,34.0,9.0,1.0,0.0,0.0,1.0,4368.413,14051.160,1
4,0.0,18.0,9.0,1.0,0.0,0.0,1.0,0.000,10740.080,1
...,...,...,...,...,...,...,...,...,...,...
292,1.0,20.0,9.0,1.0,0.0,0.0,1.0,0.000,8881.665,1
293,1.0,31.0,4.0,1.0,0.0,0.0,1.0,4023.211,7382.549,1
294,1.0,24.0,10.0,1.0,0.0,1.0,1.0,4078.152,0.000,1
295,1.0,33.0,11.0,1.0,0.0,1.0,1.0,25142.240,4181.942,1


Part b

In [4]:
Y = df["re78"]
X = df[["treated","constant"]] 

ols_model = OLS(Y, X, hasconst=True)
results = ols_model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   re78   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.525
Date:                Mon, 12 Apr 2021   Prob (F-statistic):             0.0609
Time:                        16:47:58   Log-Likelihood:                -7333.1
No. Observations:                 722   AIC:                         1.467e+04
Df Residuals:                     720   BIC:                         1.468e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
treated      886.3037    472.086      1.877      0.061     -40.526    1813.134
constant    5090.0483    302.783     16.811      0.000    4495.606    5684.491
==============================================================================
Omnibus:                      384.449   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3767.287
Skew:                           2.195   Prob(JB):                         0.00
Kurtosis:                      13.294   Cond. No.                         2.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Part c

In [5]:
tau_init = results.params["treated"]
alpha_init = results.params["constant"]
T = df["treated"]
X = df[["age", "education", "black", "hispanic","married","nodegree","re75"]] 
N = len(X)
mu_init = X.mean()

In [6]:
g = np.concatenate([np.stack([Y - alpha_init - tau_init*T, (Y - alpha_init - tau_init*T)*T]),(X-mu_init).transpose(),((X-mu_init).values*np.expand_dims(T.values, axis=1)).transpose()],axis=0).transpose()
g.shape

(722, 16)

In [7]:
# compute weights
omega = np.matmul(g.transpose(), g) / N
weights = np.linalg.inv(omega)

In [8]:
# we now construct the Jacobian

G = np.zeros([2*len(mu_init) + 2,2+len(mu_init)])
G[0,0] = 1
G[0,1] = T.mean()
G[1,0] = T.mean()
G[1,1] = (T*T).mean()
for i in range(2,len(mu_init) + 2):
    G[i,i]=1
    G[i+len(mu_init),i] = T.mean()

In [9]:
vector = np.concatenate([np.array([Y.mean(), (Y*T).mean()]), X.mean().values, (X.values*np.expand_dims(T.values,axis=1)).mean(axis=0)])

In [10]:
theta_hat = np.matmul(np.linalg.inv(np.matmul(np.matmul(G.transpose(), weights), G)), np.matmul(np.matmul(G.transpose(), weights),vector))

In [11]:
# estimated tau:
theta_hat[1]

794.3885736971342

In [12]:
covariance = np.linalg.inv(np.matmul(np.matmul(G.transpose(), np.linalg.inv(omega)), G))

In [13]:
# error
std_error = np.sqrt(covariance[1,1]/N)
std_error

480.3028362343343